# 1-3 転移学習の実装
ここでは次のような流れで PyTorch のチュートリアルに従って画像分類モデルを作成する．
1. 画像データから Dataset を作成する
1. Dataset から DataLoader を作成
1. 任意の出力ができるよう学習済みモデルの出力層を変形
1. 出力層の結合パラメータのみを学習させて転移学習を実装  

## 転移学習
学習済みモデルをベースに最後の出力層を付け替える学習手法を転移学習（transfer learning）と呼ぶ．
自前のデータに合わせて付け替えた出力層のみを学習し直す．
転移学習では少量のデータで性能の良いディープラーニングを実現しやすい．
ちなみに，入力層の近くまでパラメータを更新する場合は fine tuning と呼ばれ，1.5節で扱う．  

## フォルダの準備
make_folders_and_data_downloads.ipynb を実行する．data/hymenoptera_data だ転移学習に使用するデータ．

## 事前準備
for ループの経過時間と残り時間を計測する tqdm をインストールしておく．
```bash
$ pip install tpdm
```  

## 実装の初期設定
パッケージのインポートと乱数のシードを設定する．

In [2]:
%matplotlib inline

import glob
import json
import random
import numpy as np
import os.path as osp
from PIL import Image
from tqdm import tqdm
from matplotlib import pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import models, transforms

torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [7]:
# GPU が使えているかの確認
x = torch.randn(10)

# GPU へ送る
x = x.to("cuda")
print(x.is_cuda) # True

# CPU に戻す
x = x.to('cpu')
print(x.is_cuda)  # False

# GPU の番号を指定して送る
x = x.to("cuda:0")
print(x.is_cuda) # True

True
False
True


In [8]:
# GPU で計算するときに結果に再現性を持たせるには，次の設定が必要
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Dataset の作成